<a href="https://colab.research.google.com/github/williamvpaiva/MercadoFinanceiro/blob/main/A%C3%87%C3%95ES_VERS%C3%83O_FINAL_Analise_de_A%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3 MB 8.1 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=f06f2adde6fab5d6fca7b37b091dade8b2d20f4cfd82b43108f407dd6f0ce21b
  Stored in directory: /tmp/pip-ephem-wheel-cache-obp891ux/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly
import openpyxl

In [ ]:
%%time

# Imports
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

# Definindo características da página
url = 'https://www.fundamentus.com.br/resultado.php'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

# Fazendo a request
try:
    req = Request(url, headers = headers)
    response = urlopen(req)
    print('Foi\n\n')
    print(response.getcode())
    html = response.read()

# Tratando alguns possíveis erros
except HTTPError as e:
    print('HTTPError\n\n')
    print(response.getcode())
    print(e.status, e.reason)

except URLError as e:
    print('URLError\n\n')
    print(response.getcode())
    print(e.reason)

# Instanciando um objeto BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Pegando os nomes das colunas da tabela
colunas_names = [col.getText() for col in soup.find('table', {'id': 'resultado'}).find('thead').findAll('th')]
colunas = {i: col.getText() for i, col in enumerate(soup.find('table', {'id': 'resultado'}).find('thead').findAll('th'))}

# Criando um DataFrame com os nomes das colunas
dados = pd.DataFrame(columns=colunas_names)

# Pegando os dados da tabela por linha
for i in range(len(soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr'))):
    linha = soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr')[i].getText().split('\n')[1:]
    inserir_linha = pd.DataFrame(linha).T.rename(columns=colunas)
    dados = pd.concat([dados, inserir_linha], ignore_index=True)

# É tetra!
dados

Foi


200
CPU times: user 36.4 s, sys: 156 ms, total: 36.5 s
Wall time: 37 s


#Renomeando colunas

In [ ]:
df = dados.rename(columns={'Papel':'Codigofundo',
                        'Div.Yield':'DY',
                        'Mrg Ebit':'MrgEbit',
                        'Mrg. Líq.':'MrgLíq',
                        'Liq. Corr.':'LiqCorr',
                        'Patrim. Líq':'PatrimLíq',
                        'Dív.Brut/ Patrim.':'DívBrutPatrim',
                        'Cresc. Rec.5a':'CrescRec5a',
                        'Liq.2meses':'Liq2meses',
                        'P/L':'PL',
                        'P/VP':'PVP'
                        #'DY (6M)Média':'DY(6M)Media',
                       # 'DY Ano':'DYAno',
                        #'PatrimônioLíq.':'Patrimonio'
                        #'QuantidadeAtivos': 'ativos'
#, 'PatrimônioLíq.':'Patr Liqu'
})
df

,Codigofundo,Cotação,PL,PVP,PSR,DY,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,MrgEbit,MrgLíq,LiqCorr,ROIC,ROE,Liq2meses,PatrimLíq,DívBrutPatrim,CrescRec5a,21
0,CFLU4,"1.000,00","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","8,88%","10,72%","1,10","17,68%","32,15%","0,00","60.351.000,00","0,06","8,14%",
1,MNSA3,"0,42","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","-208,15%","-362,66%","3,63","-13,50%","145,70%","0,00","-9.105.000,00","-6,52","-41,11%",
2,CLAN3,"0,00","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","0,00%","0,00%","0,00","0,00%","-1,05%","0,00","1.012.240.000,00","0,00","-63,96%",
3,CSTB3,"150,00","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","40,85%","28,98%","2,60","22,40%","20,11%","0,00","8.420.670.000,00","0,14","31,91%",
4,MNSA4,"0,47","0,00","0,00","0,000","0,00%","0,000","0,00","0,00","0,00","0,00","0,00","-208,15%","-362,66%","3,63","-13,50%","145,70%","0,00","-9.105.000,00","-6,52","-41,11%",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,BIDI4,"23,76","3.491,83","18,91","0,000","0,03%","0,000","0,00","0,00","0,00","0,00","0,00","0,00%","0,00%","0,00","0,00%","0,54%","109.988.000,00","3.240.140.000,00","0,00","30,91%",
970,BIDI11,"72,18","3.535,92","19,15","0,000","0,03%","0,000","0,00","0,00","0,00","0,00","0,00","0,00%","0,00%","0,00","0,00%","0,54%","277.362.000,00","3.240.140.000,00","0,00","30,91%",
971,BIDI3,"24,47","3.596,17","19,47","0,000","0,03%","0,000","0,00","0,00","0,00","0,00","0,00","0,00%","0,00%","0,00","0,00%","0,54%","12.940.300,00","3.240.140.000,00","0,00","30,91%",
972,NTCO3,"54,60","4.879,16","2,62","1,938","0,00%","1,218","27,55","30,65","-4,90","33,67","15,77","6,32%","0,01%","1,18","5,05%","0,05%","406.215.000,00","28.691.300.000,00","0,49","0,00%",


# Tratando as colunas e convertendo em float

In [ ]:

df['Cotação'] = df['Cotação'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['PL'] = df['PL'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['PVP'] = df['PVP'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['PSR'] = df['PSR'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['DY'] = df['DY'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/Ativo'] = df['P/Ativo'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/Cap.Giro'] = df['P/Cap.Giro'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/EBIT'] = df['P/EBIT'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['P/Ativ Circ.Liq'] = df['P/Ativ Circ.Liq'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['EV/EBIT'] = df['EV/EBIT'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['EV/EBITDA'] = df['EV/EBITDA'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['MrgEbit'] = df['MrgEbit'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['MrgLíq'] = df['MrgLíq'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['LiqCorr'] = df['LiqCorr'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['ROIC'] = df['ROIC'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['ROE'] = df['ROE'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['Liq2meses'] = df['Liq2meses'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['PatrimLíq'] = df['PatrimLíq'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['DívBrutPatrim'] = df['DívBrutPatrim'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))
df['CrescRec5a'] = df['CrescRec5a'].apply(lambda x: float(x.replace(".","").replace(",",".").replace("%","")))	

In [ ]:
df.dropna(inplace=True) 

#Filtros

In [ ]:
# DY  maior que 2,50% ao ano
# df =  df.query('PSR <= 1.50')

In [ ]:
# Patrimonio liquido acima de 200 milhoes
df = df.query('PatrimLíq >= 2000000000')

In [ ]:
# Liquidez ultimos 2 meses maior que ou igual a 200 mil
df = df.query('Liq2meses	 >= 200000')  

In [ ]:
# DY  maior que 2,50% ao ano
df =  df.query('DY >= 5.25')

In [ ]:
# P/L menor que 15x
df =  df.query('PL <= 15.00')

In [ ]:
# Crescimento superior a 10% nos ultimos 5 anos
df = df.query('CrescRec5a >= 10.00')  

In [ ]:
# ROE maior que 15% ao ano
df = df.query('ROE >= 15.00')

In [ ]:
# Liquidez corrente maior que 1,50%
df =  df.query('LiqCorr >= 1.00')

In [ ]:
df = df.query("Codigofundo not in ['TAEE3','TAEE4','ALUP3','ALUP4','UNIP3','KLBN4','KLBN3','USIM3','RAPT4','BRKM5']")


In [ ]:
#df.to_excel('fundos_imobiliarios_.xlsx')
df.to_excel('acoes.xlsx')

In [ ]:
#df = pd.read_csv('/content/fundos_imobiliarios_.csv', sep=';', encoding='latin-1',decimal='.', thousands='.')
df = pd.read_excel('/content/acoes.xlsx')
df

,Unnamed: 0,Codigofundo,Cotação,PL,PVP,PSR,DY,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,MrgEbit,MrgLíq,LiqCorr,ROIC,ROE,Liq2meses,PatrimLíq,DívBrutPatrim,CrescRec5a,21
0,456,CYRE3,20.08,4.17,1.41,1.976,13.19,0.689,2.04,13.04,16.25,14.32,13.46,15.15,48.97,2.78,6.43,33.79,135311000,5696170000,0.47,10.74,NaN
1,485,RAPT3,12.65,5.50,2.00,0.716,5.85,0.461,1.59,5.66,-4.68,7.85,6.29,12.64,16.47,1.93,12.41,36.45,392670,2183370000,1.81,25.34,NaN
2,487,TAEE11,39.40,5.53,2.06,3.591,13.49,0.904,7.88,5.17,-2.34,7.24,7.20,69.43,64.95,2.93,18.85,37.20,64019200,6598210000,0.98,32.43,NaN
3,496,ENBR3,17.20,5.67,1.00,0.638,5.81,0.329,32.14,3.07,-0.84,5.43,4.51,20.76,12.51,1.04,11.90,17.54,49721400,10487500000,0.92,10.99,NaN
4,519,VALE3,109.70,6.36,2.78,1.972,8.06,1.198,10.47,3.25,-4.00,3.26,2.97,60.67,30.41,1.77,44.97,43.71,2616650000,208437000000,0.33,27.55,NaN
5,541,CAML3,8.90,7.14,1.22,0.412,6.88,0.443,1.19,5.51,9.14,7.69,6.02,7.47,5.77,2.19,13.47,17.09,12734600,2699680000,1.03,21.29,NaN
6,688,EGIE3,37.67,12.98,3.76,2.300,6.56,0.813,10.21,5.45,-1.47,7.81,6.72,42.17,17.72,1.53,17.62,28.99,49690800,8166660000,2.25,18.97,NaN


In [ ]:
df.dtypes

Unnamed: 0           int64
Codigofundo         object
Cotação            float64
PL                 float64
PVP                float64
PSR                float64
DY                 float64
P/Ativo            float64
P/Cap.Giro         float64
P/EBIT             float64
P/Ativ Circ.Liq    float64
EV/EBIT            float64
EV/EBITDA          float64
MrgEbit            float64
MrgLíq             float64
LiqCorr            float64
ROIC               float64
ROE                float64
Liq2meses            int64
PatrimLíq            int64
DívBrutPatrim      float64
CrescRec5a         float64
21                 float64
dtype: object

In [ ]:
df.reset_index(inplace=True)

In [ ]:
symbols = df['Codigofundo']+'.SA'
symbols

0     CYRE3.SA
1     RAPT3.SA
2    TAEE11.SA
3     ENBR3.SA
4     VALE3.SA
5     CAML3.SA
6     EGIE3.SA
Name: Codigofundo, dtype: object

In [ ]:
start = datetime(2015, 1, 1)
#end = datetime(2021, 7, 30)
end = datetime.today()- timedelta(days=1)

In [ ]:
def pega_fundos(name):
  stock = yf.download(name, start = start, end = end)
  return stock

In [ ]:
stock_dict = {}
for symbol in symbols:
  try:
    stock_dict[symbol] = pega_fundos(symbol)
  except:
    print('NÃO DEU CERTO PARA O TICKET',symbol)
print('TERMINOU')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
TERMINOU


In [ ]:
#stock_dict['JHSF3.SA']

In [ ]:
returns = pd.DataFrame()
for key in stock_dict.keys():
  returns[key] = stock_dict[key]['Adj Close'].pct_change()

returns.head()

,CYRE3.SA,RAPT3.SA,TAEE11.SA,ENBR3.SA,VALE3.SA,CAML3.SA,EGIE3.SA
Date,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,0.025837,0.006250,-0.003713,-0.040092,-0.015038,NaN,0.001227
2015-01-06,0.027985,0.086957,0.001597,0.016706,0.040076,NaN,0.020214
2015-01-07,0.026316,0.108571,0.001595,0.038732,0.036697,NaN,0.025218
2015-01-08,0.007958,-0.010309,0.009554,0.028248,0.010619,NaN,0.010248


In [ ]:
returns.describe()

,CYRE3.SA,RAPT3.SA,TAEE11.SA,ENBR3.SA,VALE3.SA,CAML3.SA,EGIE3.SA
count,1634.000000,1607.000000,1634.000000,1634.000000,1634.000000,952.000000,1634.000000
mean,0.001009,0.001516,0.001000,0.000811,0.001647,0.000337,0.000601
std,0.028777,0.033277,0.015622,0.019493,0.030836,0.020500,0.015955
min,-0.246502,-0.307566,-0.083085,-0.127682,-0.245592,-0.089247,-0.096285
25%,-0.013554,-0.013636,-0.008195,-0.009833,-0.014674,-0.011536,-0.008403
50%,0.000000,0.000000,0.000353,0.000000,0.000593,0.000000,0.000093
75%,0.016065,0.017616,0.009848,0.010881,0.016585,0.010993,0.009262
max,0.180556,0.151079,0.087322,0.155556,0.213579,0.099379,0.099224


In [ ]:
discount_dict = {}
def calcula_desconto(df):
  max_price = df['Adj Close'].describe()['75%']
  last_price = df['Adj Close'].iloc[-1]
  return ((last_price - max_price)/max_price)*100

In [ ]:
for tick in stock_dict.keys():
  try:
    discount_dict[tick] =  calcula_desconto(stock_dict[tick])
  except:
    discount_dict[tick] = 0
discount_dict

{'CAML3.SA': -10.191727996298473,
 'CYRE3.SA': -0.729088470623779,
 'EGIE3.SA': -4.221667271648467,
 'ENBR3.SA': 0.07168781753208807,
 'RAPT3.SA': 67.31517421382156,
 'TAEE11.SA': 70.59305867756972,
 'VALE3.SA': 135.44293910562095}

In [ ]:
def pega_setor(stock):
  return[x for x  in df[symbols == stock]['Codigofundo']]

stock_info_df = pd.DataFrame.from_dict(discount_dict, orient='index').reset_index()
stock_info_df.columns = ['ticker','discount']
#stock_info_df['Codigofundo'] = [pega_setor(tick)[0] for tick in stock_info_df['ticker']]

stock_info_df

,ticker,discount
0,CYRE3.SA,-0.729088
1,RAPT3.SA,67.315174
2,TAEE11.SA,70.593059
3,ENBR3.SA,0.071688
4,VALE3.SA,135.442939
5,CAML3.SA,-10.191728
6,EGIE3.SA,-4.221667


In [ ]:
std_list = []

for tick in stock_info_df['ticker']:
  std_list.append(returns[tick].std()*100)

stock_info_df['risco'] = std_list 

stock_info_df.sort_values(by = 'discount').head(10)


,ticker,discount,risco
5,CAML3.SA,-10.191728,2.049960
6,EGIE3.SA,-4.221667,1.595529
0,CYRE3.SA,-0.729088,2.877691
3,ENBR3.SA,0.071688,1.949321
1,RAPT3.SA,67.315174,3.327687
2,TAEE11.SA,70.593059,1.562181
4,VALE3.SA,135.442939,3.083582


In [ ]:
def pega_dividendos(stock):
  return[x for x  in df[symbols == stock]['DY']]
stock_info_df['DY'] = [pega_dividendos(tick)[0] for tick in stock_info_df['ticker']] 

In [ ]:
def P_L(stock):
  return[x for x  in df[symbols == stock]['PL']]
stock_info_df['PL'] = [P_L(tick)[0] for tick in stock_info_df['ticker']]


In [ ]:
def Preco_atual(stock):
  return[x for x  in df[symbols == stock]['Cotação']]
stock_info_df['Cotação'] = [Preco_atual(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def LiqCorr(stock):
  return[x for x  in df[symbols == stock]['LiqCorr']]
stock_info_df['LiqCorr'] = [LiqCorr(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def ROIC(stock):
  return[x for x  in df[symbols == stock]['ROIC']]
stock_info_df['ROIC'] = [ROIC(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def ROE(stock):
  return[x for x  in df[symbols == stock]['ROE']]
stock_info_df['ROE'] = [ROE(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def CrescRec5a(stock):
  return[x for x  in df[symbols == stock]['CrescRec5a']]
stock_info_df['CrescRec5a'] = [CrescRec5a(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def PatrimLiq(stock):
  return[x for x  in df[symbols == stock]['PatrimLíq']]
stock_info_df['PatrimLíq'] = [PatrimLiq(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def P_VP(stock):
  return[x for x  in df[symbols == stock]['PVP']]
stock_info_df['PVP'] = [P_VP(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:

def PSR(stock):
  return[x for x  in df[symbols == stock]['PSR']]
stock_info_df['PSR'] = [PSR(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def MrgEbit(stock):
  return[x for x  in df[symbols == stock]['MrgEbit']]
stock_info_df['MrgEbit'] = [MrgEbit(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def DivBrutPatrim(stock):
  return[x for x  in df[symbols == stock]['DívBrutPatrim']]
stock_info_df['DívBrutPatrim'] = [DivBrutPatrim(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def EV_EBITDA(stock):
  return[x for x  in df[symbols == stock]['EV/EBITDA']]
stock_info_df['EV/EBITDA'] = [EV_EBITDA(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def EV_EBIT(stock):
  return[x for x  in df[symbols == stock]['EV/EBIT']]
stock_info_df['EV/EBIT'] = [EV_EBIT(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def P_EBIT(stock):
  return[x for x  in df[symbols == stock]['P/EBIT']]
stock_info_df['P/EBIT'] = [P_EBIT(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def P_Ativ_Circ_Liq(stock):
  return[x for x  in df[symbols == stock]['P/Ativ Circ.Liq']]
stock_info_df['P/Ativ Circ.Liq'] = [P_Ativ_Circ_Liq(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
def P_Cap_Giro(stock):
  return[x for x  in df[symbols == stock]['P/Cap.Giro']]
stock_info_df['P/Cap.Giro'] = [P_Cap_Giro(tick)[0] for tick in stock_info_df['ticker']]

In [ ]:
##import plotly.express as px
#stock_info_df = stock_info_df.dropna()

In [ ]:
#fig = px.scatter(stock_info_df, x = 'risk', y ='discount', color = 'sector', hover_name='ticker', log_x = True)

#plotly.offline.plot(fig,filename='oportunidades.html')

In [ ]:
stock_info_df.sort_values(by = 'PL').head(20)

,ticker,discount,risco,DY,PL,Cotação,LiqCorr,ROIC,ROE,CrescRec5a,PatrimLíq,PVP,PSR,MrgEbit,DívBrutPatrim,EV/EBITDA,EV/EBIT,P/EBIT,P/Ativ Circ.Liq,P/Cap.Giro
0,CYRE3.SA,-0.729088,2.877691,13.19,4.17,20.08,2.78,6.43,33.79,10.74,5696170000,1.41,1.976,15.15,0.47,13.46,14.32,13.04,16.25,2.04
1,RAPT3.SA,67.315174,3.327687,5.85,5.50,12.65,1.93,12.41,36.45,25.34,2183370000,2.00,0.716,12.64,1.81,6.29,7.85,5.66,-4.68,1.59
2,TAEE11.SA,70.593059,1.562181,13.49,5.53,39.40,2.93,18.85,37.20,32.43,6598210000,2.06,3.591,69.43,0.98,7.20,7.24,5.17,-2.34,7.88
3,ENBR3.SA,0.071688,1.949321,5.81,5.67,17.20,1.04,11.90,17.54,10.99,10487500000,1.00,0.638,20.76,0.92,4.51,5.43,3.07,-0.84,32.14
4,VALE3.SA,135.442939,3.083582,8.06,6.36,109.70,1.77,44.97,43.71,27.55,208437000000,2.78,1.972,60.67,0.33,2.97,3.26,3.25,-4.00,10.47
5,CAML3.SA,-10.191728,2.049960,6.88,7.14,8.90,2.19,13.47,17.09,21.29,2699680000,1.22,0.412,7.47,1.03,6.02,7.69,5.51,9.14,1.19
6,EGIE3.SA,-4.221667,1.595529,6.56,12.98,37.67,1.53,17.62,28.99,18.97,8166660000,3.76,2.300,42.17,2.25,6.72,7.81,5.45,-1.47,10.21


In [ ]:
stock_info_df.dtypes

ticker              object
discount           float64
risco              float64
DY                 float64
PL                 float64
Cotação            float64
LiqCorr            float64
ROIC               float64
ROE                float64
CrescRec5a         float64
PatrimLíq            int64
PVP                float64
PSR                float64
MrgEbit            float64
DívBrutPatrim      float64
EV/EBITDA          float64
EV/EBIT            float64
P/EBIT             float64
P/Ativ Circ.Liq    float64
P/Cap.Giro         float64
dtype: object

In [ ]:
stock_info_df
#import seaborn as sns

,ticker,discount,risco,DY,PL,Cotação,LiqCorr,ROIC,ROE,CrescRec5a,PatrimLíq,PVP,PSR,MrgEbit,DívBrutPatrim,EV/EBITDA,EV/EBIT,P/EBIT,P/Ativ Circ.Liq,P/Cap.Giro
0,CYRE3.SA,-0.729088,2.877691,13.19,4.17,20.08,2.78,6.43,33.79,10.74,5696170000,1.41,1.976,15.15,0.47,13.46,14.32,13.04,16.25,2.04
1,RAPT3.SA,67.315174,3.327687,5.85,5.50,12.65,1.93,12.41,36.45,25.34,2183370000,2.00,0.716,12.64,1.81,6.29,7.85,5.66,-4.68,1.59
2,TAEE11.SA,70.593059,1.562181,13.49,5.53,39.40,2.93,18.85,37.20,32.43,6598210000,2.06,3.591,69.43,0.98,7.20,7.24,5.17,-2.34,7.88
3,ENBR3.SA,0.071688,1.949321,5.81,5.67,17.20,1.04,11.90,17.54,10.99,10487500000,1.00,0.638,20.76,0.92,4.51,5.43,3.07,-0.84,32.14
4,VALE3.SA,135.442939,3.083582,8.06,6.36,109.70,1.77,44.97,43.71,27.55,208437000000,2.78,1.972,60.67,0.33,2.97,3.26,3.25,-4.00,10.47
5,CAML3.SA,-10.191728,2.049960,6.88,7.14,8.90,2.19,13.47,17.09,21.29,2699680000,1.22,0.412,7.47,1.03,6.02,7.69,5.51,9.14,1.19
6,EGIE3.SA,-4.221667,1.595529,6.56,12.98,37.67,1.53,17.62,28.99,18.97,8166660000,3.76,2.300,42.17,2.25,6.72,7.81,5.45,-1.47,10.21


In [ ]:
#plt.figure(figsize=(16,16))
#sns.heatmap(stock_info_df.corr(),annot=True);

In [ ]:
stock_info_df.to_excel('resultado_ações.xlsx')